# Assignment 1

In this assignment, you will investigate the precision issues in computing the gradient. You will also implement a simple linear regression model using the custom autograd engine.

## Task 1: Precision issues

In [53]:
def f(x):
    return 3 * x ** 2 - 4 * x + 5

def gradient(f, x, h=0.0001):
    return (f(x + h) - f(x)) / h

gradient(f, 2)

8.000300000023941

In [54]:
# What happens if we keep decreasing h?
gradient(f, 2, h=0.0000000000000001)

################################################################################
# TODO:                                                                        #
# Why is the gradient 0?                                                       #
# If you don't know, google it!                                                #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
# ANSWER: 부동소수점 연산의 한계 컴퓨터는 유한한 비트로 실수를 표현하기 때문에, h의 값이 너무 작아지면 
# 컴퓨터는 반올림 오차로 인해 f(x+h)와 f(x)의 차이를 구분하지 못합니다. 따라서 h가 너무 작아지면 
# gradient가 0으로 수렴합니다.
# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

0.0

## Task 2: Linear Regression

Let's review the training loop of a simple linear regression model.

### Importing Libraries

In [55]:
import math
import random

In [56]:
class Tensor:
    def __init__(self, data, _children=(), _operation=''):
        self.data = data
        self._prev = set(_children)
        self.gradient = 0
        self._backward = lambda: None

    def __repr__(self):
        return f"tensor=({self.data})"

    def __add__(self, other):  # self + other
        output = Tensor(self.data + other.data, (self, other), '+')
        def _backward():
            self.gradient = 1 * output.gradient
            other.gradient = 1 * output.gradient
        output._backward = _backward
        return output

    def __mul__(self, other):  # self * other
        output = Tensor(self.data * other.data, (self, other), '*')
        def _backward():
            self.gradient = other.data * output.gradient
            other.gradient = self.data * output.gradient
        output._backward = _backward
        return output

    def tanh(self):  # tanh(self)
        output = Tensor(math.tanh(self.data), (self,), 'tanh')
        def _backward():
            self.gradient = (1.0 - math.tanh(self.data) ** 2) * output.gradient
        output._backward = _backward
        return output

    def __pow__(self, power):  # self ** power
        assert isinstance(power, (int, float)), "Power must be an int or a float"
        output = Tensor(self.data ** power, (self,), f'**{power}')
        def _backward():
            self.gradient = power * (self.data ** (power - 1)) * output.gradient
        output._backward = _backward
        return output

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.gradient = 1
        for node in reversed(topo):
            node._backward()

    def __neg__(self): # -self
        return self * Tensor(-1.0)

    def __sub__(self, other): # self - other
        return self + (-other)

### Data

**GOAL: Find the best line that fits the following data.**

![Data](../../assets/linear.png)

(Image credit: MIT 18.06)

(1, 1), (2, 2), (3, 2)

In [57]:
# Input, Target data
input = [Tensor(1), Tensor(2), Tensor(3)]
target = [Tensor(1), Tensor(2), Tensor(2)]

### Model

In [58]:
# Linear regression model
class Linear:
    def __init__(self):
        self.a = Tensor(random.uniform(-1, 1))
        self.b = Tensor(random.uniform(-1, 1))

    def __call__(self, x):
        y = self.a * x + self.b
        return y

    def parameters(self):
        return self.a, self.b

In [59]:
# Initialize the model
model = Linear()

# Example forward pass
print(f"Output: {model(input[0])}")

Output: tensor=(0.75578601517914)


### Training

Implement the training loop for the linear regression model.

Choose an appropriate learning rate.

In [60]:
lr = 2e-4  # learning rate

# Training loop
for step in range(10):
    total_loss = Tensor(0)
    
    # Forward pass
    for x, y in zip(input, target):
        ################################################################################
        # TODO:                                                                        #
        # Implement the forward pass.                                                  #
        ################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        loss = (model(x) - y) ** 2
        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        total_loss += loss

    # Backward pass
    ################################################################################
    # TODO:                                                                        #
    # Implement the backward pass.                                                 #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    for param in model.parameters():
        param.gradient = 0  # Zero the gradients
    loss.backward()
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    # Update weights
    ################################################################################
    # TODO:                                                                        #
    # Update the weights of the model using the gradients.                         #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    for param in model.parameters():
        param.data -= lr * param.gradient
    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
            
    print(f"Step: {step}, Loss: {total_loss.data}")

Step: 0, Loss: 3.5573313515027367
Step: 1, Loss: 3.531999210052825
Step: 2, Loss: 3.506863789951109
Step: 3, Loss: 3.4819235426962223
Step: 4, Loss: 3.457176932061545
Step: 5, Loss: 3.4326224339975573
Step: 6, Loss: 3.4082585365349685
Step: 7, Loss: 3.384083739688621
Step: 8, Loss: 3.3600965553621602
Step: 9, Loss: 3.3362955072534604


In [61]:
# Print the final weights of the model
print(f"y = {model.a.data}*x + {model.b.data}")

y = -0.03608990377286655*x + 0.81306253717314


## Extra Credit

Linear regression is the simplest form of neural networks. It actually does not require gradient descent to solve for the weights.

**Find a way to get the weights of the linear regression model without using gradient descent.**

In [62]:
################################################################################
# TODO:                                                                        #
# y = ax + b                                                                   #
# x, y = (1, 1), (2, 2), (3, 2)                                                #
# Find the values of a and b without using gradient descent.                   #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
import torch
x = torch.tensor([1., 2., 3.])
y = torch.tensor([1., 2., 2.])

X = torch.stack([x, torch.ones_like(x)], dim=1)

w = torch.inverse(X.T @ X) @ X.T @ y

a, b = w[0].item(), w[1].item()

print(f"a = {a}")
print(f"b = {b}")
# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

a = 0.5000004172325134
b = 0.6666655540466309
